In [6]:
import folium
import pandas as pd
from branca.element import Template, MacroElement

# --- Load Data ---
data = pd.read_csv("US_temp_rh_data.csv")

def calculate_heat_index(temp_c, rh):
    temp_f = temp_c * 9/5 + 32
    hi_f = (-42.379 + 2.04901523 * temp_f + 10.14333127 * rh
            - 0.22475541 * temp_f * rh - 6.83783e-3 * temp_f**2
            - 5.481717e-2 * rh**2 + 1.22874e-3 * temp_f**2 * rh
            + 8.5282e-4 * temp_f * rh**2 - 1.99e-6 * temp_f**2 * rh**2)
    return (hi_f - 32) * 5/9

def categorize_heat_index(hi_c):
    hi_f = hi_c * 9/5 + 32
    if 80 <= hi_f < 90:
        return 'Caution'
    elif 90 <= hi_f < 103:
        return 'Extreme Caution'
    elif 103 <= hi_f < 125:
        return 'Danger'
    elif hi_f >= 125:
        return 'Extreme Danger'
    else:
        return 'Normal'

data['heat_index'] = data.apply(lambda row: calculate_heat_index(row['TMAX'], row['RHAV']), axis=1)
data['heat_index_category'] = data['heat_index'].apply(categorize_heat_index)

# --- Color & Shape Mapping ---
color_shape = {
    'Normal': ('green', 'circle'),
    'Caution': ('#FFD70E', 'rectangle'),
    'Extreme Caution': ('orange', 'rectangle'),
    'Danger': ('red', 'triangle'),
    'Extreme Danger': ('purple', 'triangle')
}

# --- Create Map ---
m = folium.Map(location=[37.0902, -95.7129], zoom_start=5)
folium.TileLayer(opacity=0.20).add_to(m)

# --- Add Markers with Custom HTML Class for Filtering ---
for idx, row in data.iterrows():
    category = row['heat_index_category']
    color, shape = color_shape[category]
    category_class = category.replace(' ', '_')

    # Shape handling using Unicode emojis or styled CSS
    shape_html = ""

    if shape == "star":
        shape_html = f"<div class='heat-marker {category_class}' style='font-size:20px; color:{color};'>★</div>"
    elif shape == "circle":
        shape_html = f"<div class='heat-marker {category_class}' style='width:14px; height:14px; background:{color}; border-radius:50%; opacity:0.8; filter:blur(0px);'></div>"
    elif shape == "rectangle":
        shape_html = f"<div class='heat-marker {category_class}' style='width:16px; height:10px; background:{color}; opacity:0.8; filter:blur(0px); stroke: black;'></div>"
    elif shape == "triangle":
        shape_html = f"""
        <div class='heat-marker {category_class}' style='
            width: 0;
            height: 0;
            border-left: 8px solid transparent;
            border-right: 8px solid transparent;
            border-bottom: 14px solid {color};
            opacity: 0.8;
            filter: blur(0px);'>
        </div>
        """
    else:
        # fallback: simple square
        shape_html = f"<div class='heat-marker {category_class}' style='width:14px; height:14px; background:{color};'></div>"

    popup_html = f"""
        <b>{row['NAME']}</b><br>
        Heat Index: {row['heat_index']:.1f}°C<br>
        Temp (TMAX): {row['TMAX']}°C<br>
        Humidity (RHAV): {row['RHAV']}%
        <br>Category: {category}
    """

    icon = folium.DivIcon(html=shape_html)

    folium.Marker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        icon=icon,
        popup=folium.Popup(popup_html)
    ).add_to(m)

# --- HTML Controls & Legend ---
template = Template("""
{% macro html(this, kwargs) %}
<style>
  .control-box, .legend-box, .info-box {
    position: fixed;
    background: white;
    padding: 10px;
    border: 1px solid gray;
    z-index: 9999;
    font-size: 16px;
  }
  .control-box { bottom: 20px; left: 10px; }
  .legend-box { bottom: 20px; right: 10px; width: 250px; }
  .info-box { top: 10px; right: 10px; display: none; }

  .legend-shape {
    display: inline-block;
    width: 12px;
    height: 12px;
    margin-right: 5px;
  }
  .circle { border-radius: 50%; }
  .triangle { width: 0; height: 0; border-left: 6px solid transparent; border-right: 6px solid transparent; border-bottom: 12px solid currentColor; background: none; }
</style>

<div class="info-box" id="info-box">
  <h4>About This Map</h4>
  <p>Shows U.S. heat stress risk using temperature and humidity.</p>
  <p>Data is collected from National Centers for Environmental Information (NCEI), and shows the data from July, 2024.</p>
  <p>Use sliders to adjust blur, opacity, size. Toggle categories from legend.</p>
  <p><b> How to make use of this map? </b></p>
  <p> - For a general heat distribution of the US, make the shapes bigger and blur. </p>
  <p> - For a localized data, tap on the shapes to get a specific data.</p>
  <p> - You can protect your workers by this classifications, for example, if your work location is in a dangerous zone, being outside over 10 minutes could lead to heat stroke. </p>
  <p> - For newcomers, you can choose locations wisely based on real scientific data. </p>

  <p>For more information please refer to https://github.com/nemuulen/heat_stress_data_visualization.</p>
</div>

<div class="control-box">
  <label><b>Opacity</b></label><br>
  <input type="range" id="opacitySlider" min="0.1" max="1" step="0.1" value="0.8" oninput="updateOpacity(this.value)"><br><br>

  <label><b>Blur</b></label><br>
  <input type="range" id="blurSlider" min="0" max="5" step="0.5" value="0" oninput="updateBlur(this.value)"><br><br>

  <label><b>Size</b></label><br>
  <input type="range" id="sizeSlider" min="3" max="15" step="1" value="7" oninput="updateSize(this.value)"><br><br>

  <button onclick="toggleInfo()">🔎 About </button>
</div>

<div class="legend-box">
  <b>Filter Categories</b><br>
  <label><input type="checkbox" checked onchange="filterMarkers('Normal')">
    <span class="legend-shape circle" style="background: green;"></span> Normal (< 26.7°C)</label><br>

  <label><input type="checkbox" checked onchange="filterMarkers('Caution')">
    <span class="legend-shape rectangle" style="background: yellow;"></span> Caution (26.7°C – 32.2°C)</label><br>

  <label><input type="checkbox" checked onchange="filterMarkers('Extreme Caution')">
    <span class="legend-shape rectangle" style="background: orange;"></span> Extreme Caution (32.2°C – 39.4°C)</label><br>

  <label><input type="checkbox" checked onchange="filterMarkers('Danger')">
    <span class="legend-shape triangle" style="color: red;"></span> Danger (39.4°C – 51.7°C)</label><br>

  <label><input type="checkbox" checked onchange="filterMarkers('Extreme Danger')">
    <span class="legend-shape triangle" style="color: purple;"></span> Extreme Danger (≥ 51.7°C)</label><br>
</div>

<script>
function toggleInfo() {
  let box = document.getElementById("info-box");
  box.style.display = box.style.display === "none" ? "block" : "none";
}

function updateOpacity(val) {
  document.querySelectorAll('.heat-marker').forEach(el => {
    el.style.opacity = val;
  });
}

function updateBlur(val) {
  document.querySelectorAll('.heat-marker').forEach(el => {
    el.style.filter = "blur(" + val + "px)";
  });
}

function updateSize(val) {
  document.querySelectorAll('.heat-marker').forEach(el => {
    el.style.width = (val * 2) + "px";
    el.style.height = (val * 2) + "px";
  });
}

function filterMarkers(category) {
  let className = category.replaceAll(" ", "_");
  document.querySelectorAll(".heat-marker." + className).forEach(el => {
    el.style.display = (el.style.display === "none") ? "" : "none";
  });
}
</script>
{% endmacro %}
""")

macro = MacroElement()
macro._template = template
m.get_root().add_child(macro)

# --- Save Final Map ---
m.save("interactive_heat_index_map.html")
print("Map saved as interactive_heat_index_map.html")

Map saved as interactive_heat_index_map.html
